In [ ]:
import os

path = "/content/drive/MyDrive/기업연계프로젝트/datasets"
file_list = os.listdir(path)
len(file_list)

3681

In [ ]:
!pip install segmentation-models-pytorch
!pip install pytorch-lightning
!pip install torchtext
!pip install adamp
!pip install wandb
!pip install torchmetrics

In [2]:
import os 
import cv2
import torch
import torchvision
import PIL
import wandb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import torch.optim.lr_scheduler as lr_scheduler

from adamp import AdamP
from torchmetrics.functional import jaccard_index, accuracy, f1_score, precision, recall
from glob import glob
from torchvision import transforms as T
from torch.utils.data import DataLoader, Dataset
from tqdm.autonotebook import tqdm
from sklearn.model_selection import train_test_split
from ipywidgets import interact
from torchvision.transforms.functional import to_pil_image
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from pytorch_lightning.loggers import WandbLogger

# from save import save_model

In [3]:
data_dir = "/content/drive/MyDrive/기업연계프로젝트/datasets/"
images_paths = glob(data_dir + "*.jpg")
masks_paths = glob(data_dir + "*.png")

images_paths = sorted([str(p) for p in images_paths])
masks_paths = sorted([str(p) for p in masks_paths])

df = pd.DataFrame({'images': images_paths, 'masks': masks_paths})
df.head(5)

,images,masks
0,/content/drive/MyDrive/기업연계프로젝트/dat...,/content/drive/MyDrive/기업연계프로젝트/dat...
1,/content/drive/MyDrive/기업연계프로젝트/dat...,/content/drive/MyDrive/기업연계프로젝트/dat...
2,/content/drive/MyDrive/기업연계프로젝트/dat...,/content/drive/MyDrive/기업연계프로젝트/dat...
3,/content/drive/MyDrive/기업연계프로젝트/dat...,/content/drive/MyDrive/기업연계프로젝트/dat...
4,/content/drive/MyDrive/기업연계프로젝트/dat...,/content/drive/MyDrive/기업연계프로젝트/dat...


In [4]:
test_dir = "/content/drive/MyDrive/기업연계프로젝트/testsets/"
images_paths = glob(test_dir + "*.jpg")
masks_paths = glob(test_dir + "*.png")

images_paths = sorted([str(p) for p in images_paths])
masks_paths = sorted([str(p) for p in masks_paths])

test_df = pd.DataFrame({'images': images_paths, 'masks': masks_paths})
test_df.head(5)

,images,masks
0,/content/drive/MyDrive/기업연계프로젝트/tes...,/content/drive/MyDrive/기업연계프로젝트/tes...
1,/content/drive/MyDrive/기업연계프로젝트/tes...,/content/drive/MyDrive/기업연계프로젝트/tes...
2,/content/drive/MyDrive/기업연계프로젝트/tes...,/content/drive/MyDrive/기업연계프로젝트/tes...
3,/content/drive/MyDrive/기업연계프로젝트/tes...,/content/drive/MyDrive/기업연계프로젝트/tes...
4,/content/drive/MyDrive/기업연계프로젝트/tes...,/content/drive/MyDrive/기업연계프로젝트/tes...


In [5]:
train, valid = train_test_split(df, test_size=0.1, shuffle=True, random_state=42)

print(f"Train size: {len(train)}, Validation size: {len(valid)}")

Train size: 1656, Validation size: 184


In [6]:
@interact(index=(0, len(df)-1))
def show_images(index=0):
    image = cv2.imread(df.iloc[index].images)
    mask = cv2.imread(df.iloc[index].masks)

    plt.figure(figsize=(12,10))
    plt.subplot(121)
    plt.title("image")
    plt.imshow(image)
    plt.subplot(122)
    plt.title("mask")
    plt.imshow(mask)
    plt.tight_layout()

interactive(children=(IntSlider(value=0, description='index', max=1839), Output()), _dom_classes=('widget-inte…

In [7]:
import random
import albumentations as A

from albumentations.pytorch.transforms import ToTensorV2

In [20]:
base_transform = [
    # A.Normalize(
    #     mean=[0.485, 0.456, 0.406],
    #     std=[0.229, 0.224, 0.225],
    # ),
    ToTensorV2()
]

train_transform = [
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=10, 
                       border_mode=cv2.BORDER_CONSTANT, p=0.5),
    A.RandomCrop(width=320, height=320),
    A.RandomRain(brightness_coefficient=0.9, drop_width=1, blur_value=3, p=0.3),
    # A.RandomShadow(num_shadows_lower=1, num_shadows_upper=1, shadow_dimension=5, shadow_roi=(0, 0.5, 1, 1), p=1),
    # A.RandomSnow(brightness_coeff=1, snow_point_lower=0.1, snow_point_upper=0.3, p=1),
    # A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, brightness_by_max=True, p=1),
    # A.CLAHE(clip_limit=2, tile_grid_size=(8, 8), p=1),
    ]

train_transform.extend(base_transform)
train_transform = A.Compose(train_transform)

test_transform = A.Compose(base_transform)

In [29]:
class SegmentationDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        # row = self.df.loc[index].squeeze()
        # image_path = row['images']
        # mask_path = row['masks']
        image_path = self.df.iloc[index].images
        mask_path = self.df.iloc[index].masks

        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, 0) // 255
        
        if self.transform:
            transformed = self.transform(image=image, mask=mask)
            image = transformed['image']
            mask = transformed['mask']
        
        return image, mask

In [38]:
image = cv2.imread(test_df.iloc[0].images)
mask = cv2.imread(test_df.iloc[0].masks, 0) // 255

print(np.mask)

AttributeError: ignored

In [30]:
ds = SegmentationDataset(df, train_transform)
test_ds = SegmentationDataset(test_df, train_transform)

In [31]:
@interact(index=(0, len(test_df)-1))
def show_images(index=0):
    plt.figure(figsize=(20,10))
    plt.axis=("off")
    plt.subplot(121)
    plt.imshow(test_ds[index][0].cpu().detach().numpy().transpose(1,2,0))
    # plt.imshow(ds[index][0].T)
    plt.subplot(122)
    plt.imshow(np.squeeze(test_ds[index][1]).cpu().detach().numpy(), cmap='gray')
    # plt.imshow(ds[index][1].T, cmap='gray')

interactive(children=(IntSlider(value=0, description='index', max=114), Output()), _dom_classes=('widget-inter…

In [ ]:
class SegmentationModel(pl.LightningModule):
    def __init__(self, args=None, optimizer='adam', scheduler='reducelr'):
        super().__init__()
        self.model = smp.UnetPlusPlus(
            encoder_name='efficientnet-b0',  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
            encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
            in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
            classes=2,  # model output channels (number of classes in your dataset)
        )
        self.args = args
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optimizer
        self.scheduler = scheduler

    def forward(self, x):
        x = self.model(x)
        return x

    def configure_optimizers(self):
        if self.optimizer == 'adam':
            optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        elif self.args.optimizer == 'adamw':
            optimizer = torch.optim.AdamW(self.parameters(), lr=0.001)
        elif self.args.optimizer == 'adamp':
            optimizer = AdamP(self.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-2)

        if self.scheduler == "reducelr":
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, mode="max", verbose=True)
            return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val/jaccard_index_value"}

        elif self.args.scheduler == "cosineanneal":
            scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-5,
                                                                 last_epoch=-1, verbose=True)

        return {"optimizer": optimizer, "lr_scheduler": scheduler}

    def training_step(self, train_batch, batch_idx):
        image, mask = train_batch
        mask = mask.long()

        outputs = self.model(image)
        loss = self.criterion(outputs, mask)
        jaccard_index_value = jaccard_index(outputs.argmax(dim=1), mask, num_classes=2)
        acc_value = accuracy(outputs, mask)
        f1_value = f1_score(outputs, mask, num_classes=2, mdmc_average='samplewise')
        precision_value = precision(outputs, mask, average='micro', mdmc_average='samplewise')
        recall_value = recall(outputs, mask, average='micro', mdmc_average='samplewise')


        self.log('train/loss', loss, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log('train/acc', acc_value, on_epoch=True, on_step=True, prog_bar=True)
        self.log('train/jaccard_index_value', jaccard_index_value, on_epoch=True, on_step=True, prog_bar=True,
                 sync_dist=True)
        self.log('train/f1', f1_value, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log('train/precision', precision_value, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log('train/recall', recall_value, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        

        return {"loss": loss, "acc": acc_value, "jaccard_index": jaccard_index_value, "f1": f1_value, "precision": precision_value, "recall": recall_value}

    def validation_step(self, val_batch, batch_idx):
        image, mask = val_batch
        mask = mask.long()

        outputs = self.model(image)
        loss = self.criterion(outputs, mask)
        jaccard_index_value = jaccard_index(outputs.argmax(dim=1), mask, num_classes=2)
        acc_value = accuracy(outputs, mask)
        f1_value = f1_score(outputs, mask, num_classes=2, mdmc_average='samplewise')
        precision_value = precision(outputs, mask, average='micro', mdmc_average='samplewise')
        recall_value = recall(outputs, mask, average='micro', mdmc_average='samplewise')


        self.log('val/loss', loss, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log('val/acc', acc_value, on_epoch=True, on_step=True, prog_bar=True)
        self.log('val/jaccard_index_value', jaccard_index_value, on_epoch=True, on_step=True, prog_bar=True,
                 sync_dist=True)
        self.log('val/f1', f1_value, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log('val/precision', precision_value, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        self.log('val/recall', recall_value, on_epoch=True, on_step=True, prog_bar=True, sync_dist=True)
        

        return {"loss": loss, "acc": acc_value, "jaccard_index": jaccard_index_value, "f1": f1_value, "precision": precision_value, "recall": recall_value}

    def test_step(self, test_batch, batch_idx):
        image, mask = test_batch
        mask = mask.long()

        outputs = self.model(image)
        loss = self.criterion(outputs, mask)
        jaccard_index_value = jaccard_index(outputs.argmax(dim=1), mask, num_classes=2)
        acc_value = accuracy(outputs, mask)
        f1_value = f1_score(outputs.argmax(dim=1), mask, num_classes=2, mdmc_average='samplewise')
        precision_value = precision(outputs.argmax(dim=1), mask, average='micro', mdmc_average='samplewise')
        recall_value = recall(outputs, mask, average='micro', mdmc_average='samplewise')


        self.log('test/loss', loss, on_epoch=True, on_step=False, prog_bar=True, sync_dist=True)
        self.log('test/acc', acc_value, on_epoch=True, on_step=False, prog_bar=True)
        self.log('test/jaccard_index_value', jaccard_index_value, on_epoch=True, on_step=False, prog_bar=True,
                 sync_dist=True)
        self.log('test/f1', f1_value, on_epoch=True, on_step=False, prog_bar=True, sync_dist=True)
        self.log('test/precision', precision_value, on_epoch=True, on_step=False, prog_bar=True, sync_dist=True)
        self.log('test/recall', recall_value, on_epoch=True, on_step=False, prog_bar=True, sync_dist=True)

        return {"loss": loss, "acc": acc_value, "jaccard_index": jaccard_index_value, "f1": f1_value, "precision": precision_value, "recall": recall_value}


In [ ]:
kf = KFold(n_splits=5)
for idx, (train_index, val_index) in enumerate(kf.split(X=train['images'])):
    wandb_logger = WandbLogger(project="crack_test", name=f'5_fold{idx + 1:02d}', entity="leehm")
    checkpoint_callback = ModelCheckpoint(
            monitor="val/jaccard_index_value",
            dirpath="/content/drive/MyDrive/기업연계프로젝트/checkpoints",
            filename=f"5_fold{idx + 1:02d}_" + "{val/jaccard_index_value:.4f}",
            save_top_k=3,
            mode="max",
            # save_weights_only=True
        )

    early_stop_callback = EarlyStopping(monitor="val/loss", min_delta=0.00, patience=50, verbose=True,
                                            mode="min")
    model = SegmentationModel()

    train_ds = SegmentationDataset(train, train_transform)
    train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=12, num_workers=2, shuffle=True, drop_last=True)

    valid_ds = SegmentationDataset(valid, train_transform)
    val_dataloader = torch.utils.data.DataLoader(valid_ds, batch_size=12, num_workers=2)

    trainer = pl.Trainer(accelerator='gpu',
                      devices=1,
                      max_epochs=5,
                      log_every_n_steps=1,
                      logger=wandb_logger,
                      callbacks=[checkpoint_callback, early_stop_callback])
    
    trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)
    wandb.finish()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /content/drive/MyDrive/기업연계프로젝트/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | model     | UnetPlusPlus     | 6.6 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
6.6 M     Trainable params
0         Non

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric val/loss improved by 0.004 >= min_delta = 0.0. New best score: 0.079


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
train/acc_epoch,▁▆▇▇█
train/acc_step,▁▆▇▅▆▆▅▆▇▆▇▇▆▆▇▇▇▇▇▇▇▇█▆█▇▇█▇▇▆▆▅▅▇▇▇▇█▇
train/f1_epoch,▁▆▇▇█
train/f1_step,▁▆▇▅▆▆▅▆▇▆▇▇▆▆▇▇▇▇▇▇▇▇█▆█▇▇█▇▇▆▆▅▅▇▇▇▇█▇
train/jaccard_index_value_epoch,▁▅▇▇█
train/jaccard_index_value_step,▁▅▆▅▅▅▄▆▆▅▆▇▅▆▆▇▇▇▆▇▇▇▇▅▇█▇█▇▇▆▆▅▆▇█▆▇█▆
train/loss_epoch,█▃▂▂▁
train/loss_step,█▃▂▃▃▂▃▂▂▂▂▁▃▂▂▁▂▁▂▂▁▁▁▃▁▂▂▁▂▂▂▃▃▃▁▁▁▂▁▁
train/precision_epoch,▁▆▇▇█
train/precision_step,▁▆▇▅▆▆▅▆▇▆▇▇▆▆▇▇▇▇▇▇▇▇█▆█▇▇█▇▇▆▆▅▅▇▇▇▇█▇


## Test

In [ ]:
model = SegmentationModel.load_from_checkpoint('/content/drive/MyDrive/기업연계프로젝트/checkpoints/5_fold03_val/jaccard_index_value=0.8033.ckpt')
model.freeze()

In [ ]:
test_ds = SegmentationDataset(test_df, test_transform)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=1, num_workers=2)

trainer = pl.Trainer(accelerator='gpu',
                    devices=1,
                    max_epochs=5,
                    log_every_n_steps=1)
                    # logger=wandb_logger,
                    # callbacks=[checkpoint_callback, early_stop_callback])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
output = trainer.test(model=model, dataloaders=test_dataloader, verbose=1)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/acc            0.9710870981216431
         test/f1            0.9710870981216431
test/jaccard_index_value    0.7131603956222534
        test/loss           0.06886374950408936
     test/precision         0.9710870981216431
       test/recall          0.9710870981216431
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
trainer.predict(model=model, dataloaders=test_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

AttributeError: ignored